<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_0905%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
# !pip install konlpy

In [ ]:
!pip install transformers

In [ ]:
# !pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [ ]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
# DATA_PATH = "C:\python-code\project_final\data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

### 전체 데이터셋

In [ ]:
pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv").head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,저도 모르게 은근 여기 자주 시켜먹고 있었네요ㅎ \n늘 잘먹고 있습니다,5,5,5,5.0,저희 메뉴 맛있죠?ㅎㅎ 이렇게 빠져 드시는 거에요>.<\n\n그냥 일로서 만드는 게...,제육옛날도시락＋김찌 정식＋（서비스）콜라
1,1988응답하라추억의옛날도시락-방이점,김치찌개 된장찌개 잡채\n다 잘먹었습니다,5,5,5,5.0,전부 잘 드셔 주셔서 감사합니다 ><\n\n좋은 시간을 드리고 싶었어요ㅠㅠ 열심히 ...,"돼지고기김치찌개＋대접밥,우삼겹된장찌개＋대접밥,오색빛깔 잡채"
2,1988응답하라추억의옛날도시락-방이점,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,5,5,5,5.0,안녕하세요~~! :D\n\n저희 매장을 처음 찾아주신 고객님이시네요😆 처음 방문이라...,"[불맛]제육 옛날도시락,우삼겹옛날도시락＋미니냉면＋（서비스）콜라"
3,1988응답하라추억의옛날도시락-방이점,맛있습니다! 근데 국물이 안와서 아쉽습니다..,4,4,4,3.0,안녕하세요 1988응답하라추억의옛날도시락입니다.\n\n저희 측 실수로 인하여 누락이...,양념갈비 옛날도시락
4,1988응답하라추억의옛날도시락-방이점,맛있게잘먹었습니다,5,5,5,5.0,"헤헤, gh**님! 감사해요! 😊 \n\n맛있게 드셨다니 앞으로도 더 열심히 일할게...","삼겹정찬도시락,돼지고기김치찌개＋대접밥"


In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")

rename_cols = ['store','review','star','star_t','star_q','star_d','reply','food']
train.columns = rename_cols
train = train[['review','reply','star','star_t','star_q','star_d','food','store']] # 컬럼 순서 변경
train['star'].value_counts().sort_index() # 별점 분포 확인

star
1      111
2       87
3      227
4      771
5    23802
Name: count, dtype: int64

In [ ]:
train.head()

,review,reply,star,star_t,star_q,star_d,food,store
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요ㅎ \n늘 잘먹고 있습니다,저희 메뉴 맛있죠?ㅎㅎ 이렇게 빠져 드시는 거에요>.<\n\n그냥 일로서 만드는 게...,5,5,5,5.0,제육옛날도시락＋김찌 정식＋（서비스）콜라,1988응답하라추억의옛날도시락-방이점
1,김치찌개 된장찌개 잡채\n다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 ><\n\n좋은 시간을 드리고 싶었어요ㅠㅠ 열심히 ...,5,5,5,5.0,"돼지고기김치찌개＋대접밥,우삼겹된장찌개＋대접밥,오색빛깔 잡채",1988응답하라추억의옛날도시락-방이점
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,안녕하세요~~! :D\n\n저희 매장을 처음 찾아주신 고객님이시네요😆 처음 방문이라...,5,5,5,5.0,"[불맛]제육 옛날도시락,우삼겹옛날도시락＋미니냉면＋（서비스）콜라",1988응답하라추억의옛날도시락-방이점
3,맛있습니다! 근데 국물이 안와서 아쉽습니다..,안녕하세요 1988응답하라추억의옛날도시락입니다.\n\n저희 측 실수로 인하여 누락이...,4,4,4,3.0,양념갈비 옛날도시락,1988응답하라추억의옛날도시락-방이점
4,맛있게잘먹었습니다,"헤헤, gh**님! 감사해요! 😊 \n\n맛있게 드셨다니 앞으로도 더 열심히 일할게...",5,5,5,5.0,"삼겹정찬도시락,돼지고기김치찌개＋대접밥",1988응답하라추억의옛날도시락-방이점


# 데이터 전처리

## 정제

In [ ]:
def clean_text(text):
    if isinstance(text, str):
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        result = pat.sub(" ", text)
        result = re.sub(" +", " ", result)
        return result
    return ""

cols = ['review', 'reply', 'food', 'store']
for col in cols:
    train[col] = train[col].apply(clean_text)

train.head()

,review,reply,star,star_t,star_q,star_d,food,store
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5,5,5,5.0,제육옛날도시락 김찌 정식 서비스 콜라,1988응답하라추억의옛날도시락 방이점
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5,5,5,5.0,돼지고기김치찌개 대접밥 우삼겹된장찌개 대접밥 오색빛깔 잡채,1988응답하라추억의옛날도시락 방이점
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기...,5,5,5,5.0,불맛 제육 옛날도시락 우삼겹옛날도시락 미니냉면 서비스 콜라,1988응답하라추억의옛날도시락 방이점
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 1988응답하라추억의옛날도시락입니다 저희 측 실수로 인하여 누락이 되었군...,4,4,4,3.0,양념갈비 옛날도시락,1988응답하라추억의옛날도시락 방이점
4,맛있게잘먹었습니다,헤헤 gh 님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5,5,5,5.0,삼겹정찬도시락 돼지고기김치찌개 대접밥,1988응답하라추억의옛날도시락 방이점


### "~~님" → "고객님"으로 변환

In [ ]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)

### 가게명 키워드 컬럼화

In [ ]:
ygy_store_lst = sorted(list(set(train['store'])))
store_name = []
region_name = []
for store in ygy_store_lst:
    pat = re.split(' ', store)
    if len(pat) >= 2:
        region_name.append(pat[-1])
        store_name.append(" ".join(pat[:-1]))
    else:
        store_name.append(store)

In [ ]:
short_name = []
for name in store_name:
    if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
        cut = name[:6]
        short_name.append(cut)
    for length in [4, 3, 2]:
        cut = name[:length]
        if not cut[-1].isdigit(): # 마지막 텍스트가 숫자가 아닐경우에만 append
            short_name.append(cut)
short_name

['1988응답',
 '1989마라',
 '39도시락',
 '39도시',
 '39도',
 '7번가피자',
 '7번가피',
 '7번가',
 '7번',
 'GTS버',
 'GTS',
 'GT',
 'OK반점',
 'OK반',
 'OK',
 '갈비민족',
 '갈비민',
 '갈비',
 '감성커피',
 '감성커',
 '감성',
 '걸작떡볶',
 '걸작떡',
 '걸작',
 '고기듬뿍',
 '고기듬',
 '고기',
 '고기듬뿍',
 '고기듬',
 '고기',
 '고기잔뜩',
 '고기잔',
 '고기',
 '고기혁명',
 '고기혁',
 '고기',
 '고돼지',
 '고돼지',
 '고돼',
 '고씨네카',
 '고씨네',
 '고씨',
 '공주닭발',
 '공주닭',
 '공주',
 '과일에반',
 '과일에',
 '과일',
 '구구족',
 '구구족',
 '구구',
 '구래상회',
 '구래상',
 '구래',
 '국수나무',
 '국수나',
 '국수',
 '김포불닭',
 '김포불',
 '김포',
 '깨돌이김',
 '깨돌이',
 '깨돌',
 '꼬알라파',
 '꼬알라',
 '꼬알',
 '나인곱창',
 '나인곱',
 '나인',
 '냉면 숯',
 '냉면 ',
 '냉면',
 '놀부 부',
 '놀부 ',
 '놀부',
 '놀부보쌈',
 '놀부보',
 '놀부',
 '닭강점',
 '닭강점',
 '닭강',
 '닭장수섭',
 '닭장수',
 '닭장',
 '당치땡',
 '당치땡',
 '당치',
 '대찌 대',
 '대찌 ',
 '대찌',
 '대한냉면',
 '대한냉',
 '대한',
 '더티베지',
 '더티베',
 '더티',
 '더팔당매',
 '더팔당',
 '더팔',
 '덮덮밥',
 '덮덮밥',
 '덮덮',
 '도야짬뽕',
 '도야짬',
 '도야',
 '도우라이',
 '도우라',
 '도우',
 '돈까스회',
 '돈까스',
 '돈까',
 '땅끝수산',
 '땅끝수',
 '땅끝',
 '땡초곱창',
 '땡초곱',
 '땡초',
 '떡깨비',
 '떡깨비',
 '떡깨',
 '떡형',
 '떡형',
 '떡형',
 '뜸들이다',
 '뜸들이

In [ ]:
from collections import Counter

word_counts = Counter(short_name)  # 중복단어 제외(set 하면 배열이 틀어지니까)
drop_name = ['39도', '7번', 'OK', '감성', '걸작', '공주', '나인', '대한', '더티', '도우', '땅끝', '뜸들', '띵하', '명작', '미친', '배달', '부대', '비오', '손수', '손이', '쇼부', '수락', '수상', '수상한', '수제', '승도', '시민', '신자', '안녕', '안녕 ', '어시', '여왕', '오늘', '오늘은', '우리', '우리집', '우시', '유쾌', '유쾌한', '이끌', '이배', '이화', '인생', '조형', '직화', '진지', '천년', '청년', '최고', '큰아', '탐나', '팔도', '플러', '플러스', '행복', '행복한', '헬로', '혼밥', '후스']

short_name = [word for word in short_name if word_counts[word] == 1 and word not in drop_name]
print(len(short_name), short_name)

334 ['1988응답', '1989마라', '39도시락', '39도시', '7번가피자', '7번가피', '7번가', 'GTS버', 'GTS', 'GT', 'OK반점', 'OK반', '갈비민족', '갈비민', '갈비', '감성커피', '감성커', '걸작떡볶', '걸작떡', '고기잔뜩', '고기잔', '고기혁명', '고기혁', '고돼', '고씨네카', '고씨네', '고씨', '공주닭발', '공주닭', '과일에반', '과일에', '과일', '구구', '구래상회', '구래상', '구래', '국수나무', '국수나', '국수', '김포불닭', '김포불', '김포', '깨돌이김', '깨돌이', '깨돌', '꼬알라파', '꼬알라', '꼬알', '나인곱창', '나인곱', '냉면 숯', '냉면 ', '냉면', '놀부 부', '놀부 ', '놀부보쌈', '놀부보', '닭강', '닭장수섭', '닭장수', '닭장', '당치', '대찌 대', '대찌 ', '대찌', '대한냉면', '대한냉', '더티베지', '더티베', '더팔당매', '더팔당', '더팔', '덮덮', '도야짬뽕', '도야짬', '도야', '도우라이', '도우라', '돈까스회', '돈까스', '돈까', '땅끝수산', '땅끝수', '땡초곱창', '땡초곱', '땡초', '떡깨', '띵하오차', '띵하오', '라화방마', '라화방', '라화', '랜돌프뉴', '랜돌프', '랜돌', '롱쓰부대', '롱쓰부', '롱쓰', '마라의신', '마라의', '마라하오', '마라하', '맘스터치', '맘스터', '맘스', '명작파스', '명작파', '모모타코', '모모타', '모모', '모어댄버', '모어댄', '모어', '무국적식', '무국적', '무국', '미락칼국', '미락칼', '미락', '미친피자', '미친피', '밈피', '배달의쌀', '배달의', '병천토속', '병천토', '병천', '보돌미역', '보돌미', '보돌', '본점 장', '본점 ', '본점', '봉명동내', '봉명동', '봉명', '부대찌개', '부대찌', '부산오

In [ ]:
"안녕" in sorted(short_name)

False

In [ ]:
for name in short_name:
    train[name] = train['reply'].apply(lambda x: name if name in x else '') # 각 가게 별로 컬럼 추가(그렇지 않으면 덮어쓰기 됨)
    train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # store 컬럼에 컬럼 값 합치기

train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 첫 번째 문자가 콤마인 경우 제거
train = train.drop(columns=short_name) # 각 가게 별 컬럼 삭제
train.head()

,review,reply,star,star_t,star_q,star_d,food,store
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5,5,5,5.0,제육옛날도시락 김찌 정식 서비스 콜라,1988응답하라추억의옛날도시락 방이점
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5,5,5,5.0,돼지고기김치찌개 대접밥 우삼겹된장찌개 대접밥 오색빛깔 잡채,1988응답하라추억의옛날도시락 방이점
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기...,5,5,5,5.0,불맛 제육 옛날도시락 우삼겹옛날도시락 미니냉면 서비스 콜라,1988응답하라추억의옛날도시락 방이점
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 1988응답하라추억의옛날도시락입니다 저희 측 실수로 인하여 누락이 되었군...,4,4,4,3.0,양념갈비 옛날도시락,"1988응답하라추억의옛날도시락 방이점,1988응답"
4,맛있게잘먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5,5,5,5.0,삼겹정찬도시락 돼지고기김치찌개 대접밥,1988응답하라추억의옛날도시락 방이점


In [ ]:
train[train['store'].str.contains(short_name[3])].index

Index([16143, 16144, 16145, 16146, 16147, 16148, 16149, 16150, 16151, 16152,
       ...
       16251, 16252, 16253, 16254, 16255, 16256, 16257, 16258, 16259, 16260],
      dtype='int64', length=118)

In [ ]:
train.iloc[16143]

review                                   약간 탄맛이 있었지만 맛있었습니다
reply     소중한 리뷰 감사합니다 맛있는 식사시간이 될수 있게 더 노력하겠습니다 오늘도 멋진 ...
star                                                      4
star_t                                                    4
star_q                                                    4
star_d                                                  5.0
food                      TOKYO현지레시피 일본식 소고기덮밥 규동 카레 돈까스 세트
store                                                 39도시락
Name: 16143, dtype: object

### 여기까지 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv", index=False)

### 음식명 변수화

In [ ]:
# # 너무 많고 혼잡해서 사용x
# food_name = []
# for i in range(len(train['food'])):
#     names = train['food'][i].split()
#     for name in names:
#         if re.match(r'^[가-힣]', name): # 한글로 시작하는 경우만 append
#             food_name.append(name)
# food_name = sorted(list(set(food_name)))

In [ ]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv").drop_duplicates(ignore_index = True)["대표식품명"]
food_name = []
for food in food_list:
    parts = re.split('/|-|\(', food)
    food_name.append(parts[0])
food_name

['케이크',
 '치즈빵',
 '카스텔라',
 '츄러스',
 '치아바타',
 '와플',
 '찰떡빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모듬찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '소보로빵',
 '식빵',
 '초코소라빵',
 '크로와상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만두국',
 '라면',
 '만두',
 '만두국',
 '메밀국수',
 '수제비',
 '스파게티',
 '우동',
 '우동볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '스프',
 '오리죽',
 '전복죽',
 '채소죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감자국',
 '곰탕',
 '굴국',
 '김치국',
 '내장탕',
 '냉국',
 '달걀국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '무국',
 '묵국',
 '미역국',
 '배추국',
 '북어국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순대국',
 '어묵국',
 '오징어국',
 '육개장',
 '장어탕',
 '재첩국',
 '조개국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지전골',
 '동태국',
 '동

### 가게명 reply에서 삭제

In [ ]:
# store 를 리스트로 변환(반복문을 위해)
train['store'] = train['store'].str.split(',')
train['store']

0                [1988응답하라추억의옛날도시락 방이점]
1                [1988응답하라추억의옛날도시락 방이점]
2                [1988응답하라추억의옛날도시락 방이점]
3        [1988응답하라추억의옛날도시락 방이점, 1988응답]
4                [1988응답하라추억의옛날도시락 방이점]
                      ...              
24993                   [놀부 부대찌개 광주우산점]
24994                   [놀부 부대찌개 광주우산점]
24995                   [놀부 부대찌개 광주우산점]
24996    [놀부 부대찌개 광주우산점, 대찌, 부대찌개, 부대찌]
24997    [놀부 부대찌개 광주우산점, 대찌, 부대찌개, 부대찌]
Name: store, Length: 24998, dtype: object

In [ ]:
del_reply = []
for reply, store_words in zip(train['reply'], train['store']):
    reply_words = reply.split()
    new_reply_words = []
    for reply_word in reply_words:
        if not any(store_word in reply_word for store_word in store_words): # store_words에 있는 단어가 어떤 단어에도 포함되지 않으면 추가
            new_reply_words.append(reply_word)

    new_reply = ' '.join(new_reply_words)
    del_reply.append(new_reply)

del_reply

['저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제 메뉴를 누군가 드실 때 만족감 느끼시고 또 기분 좋아 지실 수 있도록 하나 하나 신경 쓰고 있어요 저희가 앞으로도 만족스러운 모습들 계속 보여 드릴게요 감사합니다',
 '전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다들 입맛이 다르시니까 늘 어느 정도는 걱정하는데 이렇게 좋은 소식 안고 와 주시니 소중합니다 앞으로도 많은 관심과 사랑 부탁드립니다',
 '안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 망설이셨을지도 모르는데 믿고 주문해 주셔서 감사드립니다 다음에도 생각나실 때 찾아주시면 맛있는 메뉴들과 기분 좋은 서비스로 보답하도록 하겠습니다 앞으로도 행복한 하루 보내시길 바랄게요',
 '안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 다음부터는 이런 일이 발생 하지 않도록 관련 사항들을 포장 시에 한번 더 꼼꼼히 체크하고 전달해 드리겠습니다 추가적으로 이 부분에 대해서 다음번에 찾아주실 때 말씀해 주시면 저희가 확인 후에 더욱더 큰 만족감을 들고 찾아 뵐 수 있도록 하겠습니다 감사합니다',
 '헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요',
 '즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사합니다 아직 부족한 점도 있을 텐데 이렇게 칭찬을 해 주셔서 무척 힘이 나는 거 같아요 오늘 늦게까지도 일 할 수 있습니다 항상 즐거운 시간을 제공해드릴 수 있는 매장이 되도록 최선을 다하겠습니다',
 '좋은 하루를 만들어 주셔서 감사합니다 맛을 알아봐 주시는 분을 만나 저희 메뉴도 행복했을 거 같아요 언제나 최고의 만족감을 드릴 수 있도록 끊임없이 공부하고 연습하겠습니다 오늘도 행복한 하루 보내시길 바랄게요',
 '찾아주셔서 감사의 말씀 드립니다 맛있게 드셔 주셔서 감사합니다 정말 좋아해 주시니 기분이 더

In [ ]:
len(train['reply']) , len(del_reply)

(24998, 24998)

In [ ]:
# train 셋에 반영
train['reply'] = del_reply
train['reply']

0        저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...
1        전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...
2        안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기...
3        안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 ...
4          헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요
                               ...                        
24993    놀부를 이용해 주셔서 감사합니다 큰 만족을 드리지 못한것 같아 죄송합니다 다만 다행...
24994    저희 놀부를 이용해 주셔서감사드립니다 집에서 조리시에도 매장과 최대한 비슷하게 드실...
24995    놀부를 이용해 주셔서 감사합니다 맛있게 드셨다니 다행입니다 저희는 정해진 레시피가 ...
24996    저희 맛있게 드셔주셔서 감사합니다 어디서 드시더라도 항상 맛있는 음식을 드실수 있도...
24997    언제나 맛있게 드셔주셔서 너무 감사드립니다 항상 한결같은 정성으로 맛있고 푸짐한 음...
Name: reply, Length: 24998, dtype: object

### 지점명 reply에서 삭제

In [ ]:
region_name

['방이점',
 '부천점',
 '용암점',
 '분당정자점',
 '본점',
 '대흥문창점',
 '성북점',
 '부대찌개',
 '부천점',
 '송파점',
 '부천시청점',
 '성신여대역점',
 '목동충촌점',
 '미금점',
 '조대점',
 '광주우산점',
 '점',
 '헬리오시티점',
 '전기구이통닭',
 '헬리오시티점',
 '정육왕국물두루치기',
 '성북점',
 '김치찜',
 '대전전민점',
 '계림점',
 '대전동구점',
 '가락점',
 '북서울꿈의숲점',
 '신중동점',
 '야탑점',
 '광주백운점',
 '마라샹궈',
 '죽전점',
 '대덕테크노벨리점',
 '성북점',
 '카페',
 '송파점',
 '신탄진점',
 '부평점',
 '송파점',
 '괴정동',
 '대전4호점',
 '분당점',
 '미금본점',
 '서현본점',
 '대전한남대점',
 '광주광천점',
 '대전역점',
 '유성점',
 '가락점',
 '부천점',
 '본점',
 '유성봉명점',
 '감자탕',
 '김치찌개',
 '관평점',
 '충남대점',
 '카마쿠라',
 '분식',
 '샐러드',
 '동대문구점',
 '미아본점',
 '혜화막내딸지점',
 '석촌점',
 '로스터스',
 '광주직영점',
 '미아본점',
 '상당구점',
 '야탑점',
 '우송대점',
 '신일점',
 '광주본점',
 '본점',
 '전문점',
 '부천점',
 '도촌점',
 '부천부평점',
 '삼전점',
 '용인수지점',
 '송파점',
 '정자본점',
 '분당미금역점',
 '부평점',
 '가오점',
 '부천점',
 '송파점',
 '판교점',
 '송파점',
 '대학로점',
 '성북점',
 '김치찌개',
 '성북점',
 '마약볶음밥',
 '서대전점',
 '',
 '파스타']

In [ ]:
for keyword in region_name:
    train['reply'] = train['reply'].str.replace(keyword, '')

- 개별 삭제(추론 결과보고 계속 추가)

In [ ]:
print(train[train['reply'].str.contains("고깃집")].index)

Index([521], dtype='int64')


In [ ]:
print(train[train['reply'].str.contains("고기맛집")].index)

remove_keywords = ["당신은 지금 치킨이","땡긴다","헬리오시티","입니다","D",
                   "정갈한 브런치","브런치","라쿤", "키토","놀부를",'놀부가','고기집','고깃집','고기맛점']
for keyword in remove_keywords:
    train['reply'] = train['reply'].str.replace(keyword, '')

train['reply'][442]

Index([842, 843, 844, 845, 846, 847, 848, 849, 850, 851,
       ...
       987, 988, 989, 990, 991, 992, 993, 994, 995, 996],
      dtype='int64', length=155)


'안녕하세요     맛있게 드셔주셔서 감사합니다 저희 치킨 드시면서 좋은 하루 되셨길 바랍니다 고객님께서 치킨이 땡기실 때마다 저희가 생각나도록 더욱 열심히 하겠습니다아 주문 정말 감사드리고 다음에 또 봐요'

In [ ]:
pattern = re.compile(r'\S+점')
train["reply"].apply(lambda x: pattern.sub('', x)) #__점 제거

0        저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...
1        전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...
2        안녕하세요  저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 ...
3        안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 ...
4          헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요
                               ...                        
24993     이용해 주셔서 감사합니다 큰 만족을 드리지 못한것 같아 죄송합니다 다만 다행이 큰...
24994    저희  이용해 주셔서감사드립니다 집에서 조리시에도 매장과 최대한 비슷하게 드실수 있...
24995     이용해 주셔서 감사합니다 맛있게 드셨다니 다행 저희는 정해진 레시피가 있어 최대한...
24996    저희 맛있게 드셔주셔서 감사합니다 어디서 드시더라도 항상 맛있는 음식을 드실수 있도...
24997    언제나 맛있게 드셔주셔서 너무 감사드립니다 항상 한결같은 정성으로 맛있고 푸짐한 음...
Name: reply, Length: 24998, dtype: object

### 음식명 reply에서 삭제

In [ ]:
del_reply = []
for reply_idx, reply in enumerate(train['reply']):
    reply_words = reply.split()
    new_reply_words = []
    for reply_word in reply_words:
        if not any(food in reply_word for food in food_name):
            new_reply_words.append(reply_word)
    new_reply = ' '.join(new_reply_words)
    train['reply'][reply_idx] = new_reply  # 원본 train['reply'][reply_idx]를 수정
    del_reply.append(new_reply)
del_reply

['저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제 메뉴를 누군가 드실 때 만족감 느끼시고 또 기분 좋아 지실 수 있도록 하나 하나 신경 쓰고 있어요 저희가 앞으로도 만족스러운 모습들 계속 보여 드릴게요 감사합니다',
 '전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다들 입맛이 다르시니까 늘 어느 정도는 걱정하는데 이렇게 좋은 소식 안고 와 주시니 소중합니다 앞으로도 많은 관심과 사랑 부탁드립니다',
 '안녕하세요 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 망설이셨을지도 모르는데 믿고 주문해 주셔서 감사드립니다 다음에도 생각나실 때 찾아주시면 맛있는 메뉴들과 기분 좋은 서비스로 보답하도록 하겠습니다 앞으로도 행복한 하루 보내시길 바랄게요',
 '안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 다음부터는 이런 일이 발생 하지 않도록 관련 사항들을 포장 시에 더 꼼꼼히 체크하고 전달해 드리겠습니다 추가적으로 이 부분에 대해서 찾아주실 때 말씀해 주시면 저희가 확인 후에 더욱더 큰 만족감을 들고 찾아 뵐 수 있도록 하겠습니다 감사합니다',
 '헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요',
 '즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사합니다 아직 부족한 도 있을 텐데 이렇게 칭찬을 해 주셔서 무척 힘이 나는 거 같아요 오늘 늦게까지도 일 할 수 있습니다 항상 즐거운 시간을 제공해드릴 수 있는 매장이 되도록 최선을 다하겠습니다',
 '좋은 하루를 만들어 주셔서 감사합니다 맛을 알아봐 주시는 분을 만나 저희 메뉴도 행복했을 거 같아요 언제나 최고의 만족감을 드릴 수 있도록 끊임없이 공부하고 연습하겠습니다 오늘도 행복한 하루 보내시길 바랄게요',
 '찾아주셔서 감사의 말씀 드립니다 맛있게 드셔 주셔서 감사합니다 정말 좋아해 주시니 기분이 더 좋은 거 같아요 다

In [ ]:
"국밥" in food_name

True

In [ ]:
print(train[train['reply'].str.contains("국밥")].index)

Index([], dtype='int64')


In [ ]:
train['reply'][642]

'고객님 사랑하는 저희 매장 찐팬 고객니이이이임 잘 지내셨나요 정말 오랜만에 뵙네요 오랜만에 생각해주시고 다시 찾아주셔서 정말 감사드려용 꾸벅 맛있게 드셨다니 너어어어무 기쁘네용 고객님 식탁에 다시 맛있는 한 끼 올릴 수 있길 바라며 지금처럼 언제나 변함 없는 맛으로 기다리고 있을게용 약속 바쁘시더라도 건강 챙기시면서 일 하셨으면 좋겠어요 저희 48시간 푹 끓여 고아낸 육수와 당일 도축한 신선한 고기들을 직접 손질하고 삶아 정말 정성을 다해 조리하고 있어요 믿고 주문해주셔서 감사드립니다 다음 주문도 기대해주세요 고객님들의 의견을 새겨 듣고 계속 연마하여 더 나은 맛과 좋은 서비스로 보답해드릴게요 소중한 시간 내어 리뷰 남겨주셔서 다시 감사드립니다 또 뵙는 날까지 기다릴게요'

### 여기까지 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove.csv", index=False)

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove.csv")

# 텍스트 Paraphrase
```
https://huggingface.co/lcw99/t5-base-korean-paraphrase
```

In [ ]:
train['star'].value_counts().sort_index() # 별점 분포 확인

star
1      111
2       87
3      227
4      771
5    23802
Name: count, dtype: int64

In [ ]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

## 하이퍼파라미터 튜닝

In [ ]:
star_1 = train[train['star'] == 1]
star_2 = train[train['star'] == 2]
star_3 = train[train['star'] == 3]
star_4 = train[train['star'] == 4]

train_star = pd.concat([star_1, star_2, star_3, star_4], ignore_index=True)
train_star

,review,reply,star,star_t,star_q,star_d,food,store
0,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다 배달이 20분 넘게 안오길래 전...,안녕하세요 주신 리뷰 잘 읽었습니다 뒤돌아 보니 저희 응대에 미흡했던 이 있던 것 ...,1,1,1,1.0,강추 우삼겹옛날도시락,"['1988응답하라추억의옛날도시락 방이점', '1988응답']"
1,만들어 놓은지 한참 된 커피 배달 예정 시간 25분이 지났어요,안녕하세요 배달이 늦어져 죄송합니다 한파로 길이 얼어서 고객님들이 상황이 여유치가 ...,1,1,1,1.0,NEW 레몬 글레이즈드 크로넛 행사중가격 디카페인 스페셜티 솔트캬라멜라떼,['워커스커피 로스터스']
2,,사이드만 잔뜩 시키고 1,1,1,5,5.0,김말이 2개 옛날핫도그 1개 부산오뎅탕 참치주먹밥,['떡깨비 가락점']
3,밥이 쉬었어요 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가 나서...,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요 진짜 먼 정신이 이상한 사람인가 하루에 ...,1,1,5,5.0,실속순대세트 참치주먹밥,['떡깨비 가락점']
4,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주문어플에 렉이 자주 있어서 여러가지로 혼돈이 있어 정말 죄송하게 되었습니...,1,3,3,3.0,NaN,['떡깨비 가락점']
...,...,...,...,...,...,...,...,...
1191,맛있게 잘먹었어요,이용해 주셔서 감사합니다 맛있는 식사되셨기를 바랍니다,4,3,4,3.0,NaN,"['놀부 부대찌개 광주우산점', '대찌', '부대찌개', '부대찌']"
1192,맛닜어용 괜탆네요,이용해 주셔서 감사합니다 맛있는 식사 되셨다니 다행 혹시 배달에 뭔가 부족한 이 있...,4,4,4,2.0,NaN,['놀부 부대찌개 광주우산점']
1193,맛있어요,이용해주셔서 감사합니다 더불어 맛있게 드셔주신 또한 감사드립니다 늘 좋은 음식으로 ...,4,4,4,5.0,NaN,"['놀부 부대찌개 광주우산점', '대찌', '부대찌개', '부대찌']"
1194,맛잇어요 끓여먹기는 귀찮아도 간단하게 먹기조아요,이용해 주셔서 감사합니다 아무래도 매장에서 드시는 것보다는 조금 미흡할수 있으나 최...,4,4,5,4.0,NaN,"['놀부 부대찌개 광주우산점', '대찌', '부대찌개', '부대찌']"


### BLEU 계산 함수

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

### review

In [ ]:
# import optuna
# from transformers import pipeline

# def objective(trial):
#     hp = {
#         "num_beams" : trial.suggest_int("num_beams", 1, 10),
#         "num_return_sequences" : trial.suggest_int("num_return_sequences", 1, 5),
#         "no_repeat_ngram_size" : trial.suggest_int("no_repeat_ngram_size", 1, 5),
#         "top_k" : trial.suggest_int("top_k", 10, 100),
#         "top_p" : trial.suggest_float("top_p", 0.8, 1.0)
#     }
#     # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
#     if hp["num_return_sequences"] > hp["num_beams"]:
#         hp["num_return_sequences"] = hp["num_beams"]

#     pipe = pipeline(
#         "text2text-generation",
#         model="lcw99/t5-base-korean-paraphrase",
#         device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     )

#     bleu_scores = []
#     for prompt in tqdm(train_star["review"]): # star1~4까지
#         generated_text = pipe(prompt, max_length=128, **hp, early_stopping=True)

#         bleu_score = calculate_bleu(prompt, generated_text[0]['generated_text'])
#         bleu_scores.append(bleu_score)

#     # 모든 입력 텍스트에 대한 BLEU 점수 평균 계산 및 반환
#     bleu_score_avg = sum(bleu_scores) / len(bleu_scores)
#     return bleu_score_avg

In [ ]:
# # 하이퍼파라미터 최적화
# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(sampler=sampler, direction="maximize")
# study.optimize(objective, n_trials=12)

# # 최적 값 출력
# best_bleu_score = study.best_value
# best_params = study.best_params
# print("best_bleu_score:", best_bleu_score)
# print("best_params:", best_params)

[I 2023-09-04 18:21:25,156] A new study created in memory with name: no-name-88447d6d-9e54-42fa-ad15-c88fcd87f12d


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 18:57:51,133] Trial 0 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}. Best is trial 0 with value: 0.285731536841964.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 19:25:28,423] Trial 1 finished with value: 0.28140373147755066 and parameters: {'num_beams': 2, 'num_return_sequences': 1, 'no_repeat_ngram_size': 5, 'top_k': 64, 'top_p': 0.941614515559209}. Best is trial 0 with value: 0.285731536841964.


...

  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-04 23:49:39,909] Trial 10 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 11, 'top_p': 0.8030735474656209}. Best is trial 0 with value: 0.285731536841964.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 00:20:59,956] Trial 11 finished with value: 0.285731536841964 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 17, 'top_p': 0.8007473955626367}. Best is trial 0 with value: 0.285731536841964.


best_bleu_score: 0.285731536841964
best_params: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}


### reply

In [ ]:
# import optuna
# from transformers import pipeline

# def objective(trial):
#     hp = {
#         "num_beams" : trial.suggest_int("num_beams", 1, 10),
#         "num_return_sequences" : trial.suggest_int("num_return_sequences", 1, 5),
#         "no_repeat_ngram_size" : trial.suggest_int("no_repeat_ngram_size", 1, 5),
#         "top_k" : trial.suggest_int("top_k", 10, 100),
#         "top_p" : trial.suggest_float("top_p", 0.8, 1.0)
#     }
#     # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
#     if hp["num_return_sequences"] > hp["num_beams"]:
#         hp["num_return_sequences"] = hp["num_beams"]

#     pipe = pipeline(
#         "text2text-generation",
#         model="lcw99/t5-base-korean-paraphrase",
#         device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     )

#     bleu_scores = []
#     for prompt in tqdm(train_star["reply"]): # star1~4까지
#         generated_text = pipe(prompt, max_length=128, **hp, early_stopping=True)

#         bleu_score = calculate_bleu(prompt, generated_text[0]['generated_text'])
#         bleu_scores.append(bleu_score)

#     # 모든 입력 텍스트에 대한 BLEU 점수 평균 계산 및 반환
#     bleu_score_avg = sum(bleu_scores) / len(bleu_scores)
#     return bleu_score_avg

In [ ]:
# # 하이퍼파라미터 최적화
# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(sampler=sampler, direction="maximize")
# study.optimize(objective, n_trials=12)

# # 최적 값 출력
# best_bleu_score = study.best_value
# best_params = study.best_params
# print("best_bleu_score:", best_bleu_score)
# print("best_params:", best_params)

[I 2023-09-05 09:06:47,480] A new study created in memory with name: no-name-d76cd6a4-4637-44d8-9c4e-b723d6953cec


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 09:55:30,577] Trial 0 finished with value: 0.21087345372650515 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}. Best is trial 0 with value: 0.21087345372650515.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 10:37:42,767] Trial 1 finished with value: 0.1987671035816911 and parameters: {'num_beams': 2, 'num_return_sequences': 1, 'no_repeat_ngram_size': 5, 'top_k': 64, 'top_p': 0.941614515559209}. Best is trial 0 with value: 0.21087345372650515.


...

  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 17:06:57,926] Trial 10 finished with value: 0.21087345372650515 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 11, 'top_p': 0.8030735474656209}. Best is trial 0 with value: 0.21087345372650515.


  0%|          | 0/726 [00:00<?, ?it/s]

[I 2023-09-05 17:56:53,340] Trial 11 finished with value: 0.21087345372650515 and parameters: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 17, 'top_p': 0.8007473955626367}. Best is trial 0 with value: 0.21087345372650515.


best_bleu_score: 0.21087345372650515
best_params: {'num_beams': 4, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}


# 오버샘플링

In [ ]:
review_best_params = {'num_beams': 5, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}

over_review = []
for prompt in tqdm(train_star['review']):
    generated_text = pipe(prompt, max_length=128, **review_best_params, early_stopping=True)
    generated_text = generated_text[0]['generated_text']
    over_review.append(generated_text)

over_review

  0%|          | 0/1196 [00:00<?, ?it/s]

['아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서 안 된다고 하시는데 제가 그 말을 이해할 수 없습니다.',
 '커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만들어 놓은 지 한참이 되었어요',
 '이 씨 부부의 경우 임신 8개월째로 배 속 아기가 근심되어 허리를 약간 굽히고 한 손으로 배를 가린 채 전동차에 올랐다.',
 '김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었어요 나머지는 잘 먹었어요',
 '처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역시나 문 연 지 한 시간 만에 만석, 남은 음식은 포장해 놓은 채로 남았고 그 자리에서 바로 처리해 주는 서비스에 감동했다.',
 '이 지역 사람이 아니라 현재위치로 설정해 놓은 상세 주소에 의하면 다른 배달 음식점은 20분도 채 안 걸려 도착했는데 여기만 유독 오래 걸리는 거예요.',
 '다 떠나서 게 몸에 살이 거의 없어요 2개는 사진 첨부 아예 살이 없는데 다음부터는 무조건 먹을거 같아요 맛도',
 '한 점 올렸다고 글 지우고 진상짓 하는 거 아닙니다 그냥 더 묻지 말아 주세요 양이 적기도 하고 맛도 적어요',
 '닭고기는 냄새가 엄청 나고 상추는 아삭아삭하고 고기는 부들부들한데 식감이 전혀 부드럽지 않아요 47000원짜리 치고 이렇게 질기진 않아요',
 '도보로 10분 거리에 위치한 익스프래스에서 12시 47분에 주문한 물건은 12시가 되기 7분 전에 도착했고 음식도 모두 식어 있었습니다.',
 '딴 사람들은 다 모이만큼 먹는데 나만 안 그런가 봄 내가 시킨 음식은 다 내 입맛에 맞는 것인데 말이지',
 '쌀국수 양은 많은데 고기가 너무 질겨서 한 입 먹고 안 먹을 것 같아요 사장님 죄송해요',
 '배가 고팠기 때문에 억지로 음식을 먹었지만 면을 씹으면 쓴 맛이 나고 안 익은 밀가루 면 같은 텁텁한 느낌이 나서 다 못 먹었어요',
 '보통은 새우라도 있는데 정말 아무

In [ ]:
reply_best_params = {'num_beams': 5, 'num_return_sequences': 5, 'no_repeat_ngram_size': 4, 'top_k': 64, 'top_p': 0.8312037280884873}

over_reply = []
for prompt in tqdm(train_star['reply']):
    if isinstance(prompt, str):
        generated_text = pipe(prompt, max_length=128, **reply_best_params, early_stopping=True)
        generated_text = generated_text[0]['generated_text']
        over_reply.append(generated_text)
    else:
        pass # 결측치가 있었네.. 건너뛰기
over_reply

  0%|          | 0/1196 [00:00<?, ?it/s]

['응대 방법을 다시 한번 점검해 보겠습니다 고객님이 불편을 드려 죄송합니다 다음에 또 좋은 내용으로 찾아오겠습니다 감사합니다',
 '추운 날씨로 인해 길이 꽁꽁 얼어 고객님들이 많이 힘들어 하셨는데 다시 주문해 주셔서 감사합니다.',
 '사이드만 잔뜩 시키고 1+1 사이드만 시키는 그런 스타일이에요.',
 '이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 말이라고 생각합니다.',
 '잠실에서 기분이 나쁘셨었는데 저희가 또 실수를 했네요 사정을 다시 말씀 드리겠습니다.',
 '본인이 직접 말하지 않는 이상 누가 시켜서 하는 일은 없습니다 본인이 알고 있는 사실에 대해 솔직히 말씀해 주세요',
 '많이 아쉬우셨다면 믿고 주문해 주셨는데 만족스러운 식사를 재공하지 못해 너무나 죄송합니다 꽂게 원물상태 작업관련 여러가지 검토해보겠습니다 부족한 부분들 개선해서 더욱더 좋은',
 '항상 고객님들이 기분 좋게 식사하실 수 있도록 노력하고 있는데 아쉬운 부분이 많았던 것 같습니다 다음에는 더 좋은 요리로 찾아 뵙겠습니다.',
 '저희 매장을 찾아와 주신 분들께 감사의 말씀을 전하며 더 좋은 서비스와 좋은 제품으로 보답하겠습니다.',
 '다음에도 이런 일이 생기면 여기에 바로 신고하시고 불편을 드린 분은 저희에게 메일을 주세요 그럼 처리해 드리겠습니다.',
 '구정 연휴에 날씨도 너무 추웠지만 배달이 좀 많이 밀려 있었습니다 죄송합니다 그럼 빠른 배달 해 드리도록 하겠습니다.',
 '항상 여러분의 성원에 항상 감사의 마음을 전하며 항상 노력하는 그런 업체가 되고 싶습니다.',
 '죄송합니다 고객님 덕분에 맛있는 요리 만들어 드릴 수 있었는데 다음 날 바로 매장에서 재교육 하겠습니다 감사합니다',
 '고객님 죄송합니다 주문할 때 파인애플 안 해주셨는데 죄송합니다 죄송합니다 이 메뉴는 해물은 안 들어가고 주로 과일만 넣는데 고객님 만족을 못해 드려서 죄송합니다',
 '고객님께 바로 전화 드릴 수 없어서 죄송합니다 오늘 직원들과 현장 교육을 하면

- 합치기

In [ ]:
train_over_review = pd.DataFrame({"review":over_review})
train_over_reply = pd.DataFrame({"reply":over_reply})

# concat
train_over = pd.concat([train_over_review, train_over_reply, train_star["star"]], axis=1)
train_over

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,응대 방법을 다시 한번 점검해 보겠습니다 고객님이 불편을 드려 죄송합니다 다음에 또...,1
1,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,추운 날씨로 인해 길이 꽁꽁 얼어 고객님들이 많이 힘들어 하셨는데 다시 주문해 주셔...,1
2,이 씨 부부의 경우 임신 8개월째로 배 속 아기가 근심되어 허리를 약간 굽히고 한 ...,사이드만 잔뜩 시키고 1+1 사이드만 시키는 그런 스타일이에요.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실에서 기분이 나쁘셨었는데 저희가 또 실수를 했네요 사정을 다시 말씀 드리겠습니다.,1
...,...,...,...
1191,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
1192,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4
1193,진짜 맛있어요 특히 간장 양념이 너무 달지 않고 새콤해서 더 맛있게 느껴져요,많은 사랑 주셔서 감사드립니다 감사합니다 말씀 드릴 것이 많은데도 불구하고 부족한 ...,4
1194,간편하게 끓여서 먹기도 하고 찌개 끓이는 것도 귀찮아도 간편해서 좋아요.,NaN,4


- 결측치 확인 및 제거

In [ ]:
train.isnull().sum()

review       0
reply        0
star         0
star_t       0
star_q       0
star_d       0
food      1011
store        0
dtype: int64

In [ ]:
idx_null = train[train['reply'].isnull()].index
train = train.drop(idx_null, axis=0)
train.isnull().sum()

review       0
reply        0
star         0
star_t       0
star_q       0
star_d       0
food      1011
store        0
dtype: int64

- 저장

In [ ]:
train_over.to_csv(f'{DATA_PATH}train_over_0905.csv', index=False, encoding = "utf-8-sig")

- 불러오기

In [ ]:
train_over = pd.read_csv(f'{DATA_PATH}train_over_0905.csv')
train_over

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.,1
1,양이 2만원씩이나하는데 장난으로하는것도 아니고 진짜 이거 2만원주고 사서 먹을 정도...,저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고...,1
2,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,한파로 길이 꽁꽁 얼어 고객님들이 불편을 드려 죄송했는데 주문해 주셔서 감사합니다.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실점에서 기분이 상하셨었는데 저희가 다시 실수를 한 것 같은데 다시 한번 연락드리...,1
...,...,...,...
721,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
722,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
723,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
724,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4
